In [7]:
import numpy as np
import pymc3 as pm
import pandas as pd

from aesara import tensor as at

from pymc3.math import logsumexp

In [2]:
K = 47

sunspot_df = pd.read_csv(
    pm.get_data("sunspot.csv"), sep=";", names=["time", "sunspot.year"], usecols=[0, 1]
)
sunspot_df.head()

,time,sunspot.year
0,1700.5,8.3
1,1701.5,18.3
2,1702.5,26.7
3,1703.5,38.3
4,1704.5,60.0


In [16]:
def stick_breaking(beta):
    portion_remaining = at.concatenate([[1], at.cumprod(1 - beta)[:-1]])

    return beta * portion_remaining


def mixture_logp(weights, comp_dist, atoms, obs):
    # comp_dist is something like lambda obs, atoms: pm.Normal.logp(obs, atoms, 1).eval()

    atoms_shape = atoms.shape.eval()
    
    try:
        obs_shape = obs.shape.eval()
    except AttributeError:
        obs_shape = obs.shape

    # assert atoms.shape == weights.shape

    weights = at.broadcast_to(weights, shape=obs_shape + atoms_shape)
    log_weights = (at.log(weights)).T
    atoms_logp = comp_dist(
        at.broadcast_to(obs, atoms_shape + obs_shape),
        at.broadcast_to(atoms, obs_shape + atoms_shape).T,
    )

    return logsumexp(log_weights + atoms_logp)

Taken from https://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/blob/master/notebooks/Section5_2-Dirichlet-Processes.ipynb

In [22]:
with pm.Model() as model:
    alpha = pm.Gamma("alpha", 1.0, 1.0)
    beta = pm.Beta("beta", 1, alpha, shape=K)
    w = pm.Deterministic("w", stick_breaking(beta))
    # Gamma is conjugate prior to Poisson
    lambda_ = pm.Gamma("lambda_", 300.0, 2.0, shape=K)
    obs = pm.Potential(
        "obs", 
        mixture_logp(w, lambda obs, lam: pm.Poisson.logp(obs, lam), lambda_, sunspot_df["sunspot.year"].values)
    )
    
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: BroadcastTo(TensorConstant{[  8.3  18..9.8  21.7]}, TensorConstant{365})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 2014, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 1187, in transform
    return self.fn(*args, **kwargs)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/tensor/basic_opt.py", line 4356, in constant_folding
    required = thunk()
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/op.py", line 496, in r

ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: BroadcastTo(TensorConstant{[  8.3  18..9.8  21.7]}, TensorConstant{365})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 2014, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 1187, in transform
    return self.fn(*args, **kwargs)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/tensor/basic_opt.py", line 4356, in constant_folding
    required = thunk()
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/op.py", line 496, in rval
    r = p(n, [x[0] for x in i], o)
  File "/Users/larryshamalama/anacond

ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: BroadcastTo(TensorConstant{[  8.3  18..9.8  21.7]}, TensorConstant{365})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 2014, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/opt.py", line 1187, in transform
    return self.fn(*args, **kwargs)
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/tensor/basic_opt.py", line 4356, in constant_folding
    required = thunk()
  File "/Users/larryshamalama/anaconda3/envs/pymc3-dev-py38/lib/python3.8/site-packages/aesara/graph/op.py", line 496, in rval
    r = p(n, [x[0] for x in i], o)
  File "/Users/larryshamalama/anacond

Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, beta, lambda_]


RuntimeError: Chain 0 failed.